Linking qgrid to other visualizations using event callbacks
===============================================

The following examples use qgrid's new "events API" that is only available in qgrid version 1.0.3 or newer. Specifically this notebook makes use of the ``on`` methods that qgrid provides for attaching event handlers. Event handlers can be attached using the ``qgrid.on`` method to listen for events on all widgets, or using the ``QgridWidget.on`` method the  to listen for events on individual QgridWidget instances. 

## Example 1 - Filter matplotlib scatter plot using qgrid
In this example you'll see that by listening for a QgridWidget instances's ``filter_changed`` event, we can use qgrid to filter the data that's being visualized by another control in the notebook, in this case a matplotlib scatter plot. 

This capability allows you to filter a visualization by ANY field in the underlying DataFrame, including fields that are not used to generate the visualization. This enables interesting workflows like using a Categorical column to mark a particular row of the DataFrame with as "bad", setting a filter to hide "bad" rows in the qgrid instance, and then seeing the "bad" rows also disappear from any visualizations that the qgrid instance is hooked up to.

The try out a simple example of using qgrid to filter another visualization, first execute the following two cells. Once you do that you should see a qgrid instance and a matplotlib scatter plot showing the same data as the qgrid instance. Next, set some filters on the columns of the qgrid instance and watch what happens to the scatter plot. You should see it update immediately to reflect the filtering changes.

In [ ]:
import numpy as np
import pandas as pd
import qgrid
randn = np.random.randn
df_types = pd.DataFrame({
    'A' : pd.Series(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08', '2013-01-09'],index=list(range(9)),dtype='datetime64[ns]'),
    'B' : pd.Series(randn(9),index=list(range(9)),dtype='float32'),
    'C' : pd.Categorical(["washington", "adams", "washington", "madison", "lincoln","jefferson", "hamilton", "roosevelt", "kennedy"]),
    'D' : ["foo", "bar", "buzz", "bippity","boppity", "foo", "foo", "bar", "zoo"] })
df_types['E'] = df_types['D'] != 'foo'
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=True)
qgrid_widget

In [ ]:
col_opts = { 
    'editable': False,
    'toolTip': "Not editable"
}

col_defs = {
    'B': {
        'editable': True,
        'toolTip': "Editable"
    },
    'E': {
        'editable': True,
        'toolTip': "Also editable",
        'width': 30
    }
}
qgrid_widget = qgrid.show_grid(df_types, column_options=col_opts, column_definitions=col_defs, show_toolbar=True)
qgrid_widget

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
from traitlets import All

n = 50

qgrid_df = qgrid_widget.get_changed_df()
x = qgrid_df.index
y = qgrid_df['B']

fig, ax = plt.subplots()
fit = np.polyfit(x, y, deg=1)
line, = ax.plot(x, fit[0] * x + fit[1], color='red')
scatter, = ax.plot(x,y,ms=8,color='b',marker='o',ls='')

def handle_qgrid_event(event, widget):
    qgrid_df = qgrid_widget.get_changed_df()
    x = qgrid_df.index
    y = qgrid_df['B']
    fit = np.polyfit(x, y, deg=1)
    line.set_data(x, fit[0] * x + fit[1])
    fig.canvas.draw()
    scatter.set_data(x, y)
    fig.canvas.draw()

qgrid_widget.on(All, handle_qgrid_event)


## Example 2 - Logging all events
In this example we'll see how you can listen for events from any qgrid instance using the `qgrid.on` method.

Execute the following two cells. The first cell will create an empty output widget, and the second cell use the `on` method to listen for all events from all qgrid instances. Once the cells are executed, try interacting with any of the qgrid instances you created earlier in the notebook (i.e. by sorting, filtering, scrolling, etc).  Then scroll back down to this output widget and you'll notice that any actions that you took got printed to the output widget.

In practice you'd probably want to do something more interesting than just print these events to the notebook, such as  log the events to a service that you use to track user interactions.  This is just a proof-of-concept to show which events are available.

In [ ]:
import ipywidgets as widgets
user_interactions = widgets.Output(layout={'border': '1px solid black'})
user_interactions

In [ ]:
import qgrid
from traitlets import All
from pprint import pprint

def handle_all_events(event, qgrid_widget):
    if event['name'] != 'json_updated':
        with user_interactions:
            pprint(event)

qgrid.on(All, handle_all_events)

In [ ]:
user_interactions.clear_output()

## Example 3 - Highlight visible rows in matplotlib price chart using qgrid
This example is basically a repeat of Example 1, except with a line chart of some pricing data for the S&P 500 instead of a scatter plot.

First, execute the following two cells. Once you do that you should see a qgrid instance and a matplotlib line chart showing the same data as the qgrid instance. Next, set some filters on the columns of the qgrid instance and watch what happens to the line chart (it should update immediately to reflect the filtering changes).

In [ ]:
import pandas as pd
import numpy as np
import qgrid
randn = np.random.randn

# Get a pandas DataFrame containing the daily prices for the S&P 500 from 1/1/2014 - 1/1/2017
from pandas_datareader.data import DataReader
spy = DataReader(
    'SPY',
    'yahoo',
    pd.Timestamp('2014-01-01'),  
    pd.Timestamp('2017-01-01'),
)
# Tell qgrid to automatically render all DataFrames and Series as qgrids.
qgrid.enable()

# Render the DataFrame as a qgrid automatically
spy_qgrid = qgrid.show_grid(spy, grid_options={'forceFitColumns': False})
spy_qgrid

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from traitlets import All

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
yearsFmt = mdates.DateFormatter('%Y')

n = 50
qgrid_spy_df = spy_qgrid.get_changed_df()
x = qgrid_spy_df.index
y = qgrid_spy_df['Close']

fig, ax = plt.subplots()
vp_start = mdates.date2num(qgrid_spy_df.iloc[spy_qgrid._viewport_range[0]].name)
vp_end = mdates.date2num(qgrid_spy_df.iloc[spy_qgrid._viewport_range[1]].name)
vp_span = plt.axvspan(vp_start, vp_end, facecolor='g', alpha=0.2)
line, = ax.plot(x, y)

# format the ticks
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(yearsFmt)
ax.xaxis.set_minor_locator(months)

datemin = datetime.date(x.min().year, 1, 1)
datemax = datetime.date(x.max().year + 1, 1, 1)
ax.set_xlim(datemin, datemax)

# format the coords message box
def price(x):
    return '$%1.2f' % x
ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')
ax.format_ydata = price
ax.grid(True)

# rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them
fig.autofmt_xdate()

def handle_viewport_changed(event, widget):
    qgrid_spy_df = widget.get_changed_df()
    viewport = spy_qgrid._viewport_range
    
    vp_start = mdates.date2num(qgrid_spy_df.iloc[viewport[0]].name)
    vp_end = mdates.date2num(qgrid_spy_df.iloc[viewport[1]].name)

    vp_span.set_xy([(vp_start, 0), (vp_start, 1), (vp_end, 1), (vp_end ,0)])
    
    
spy_qgrid.on('viewport_changed', handle_viewport_changed)